# Run a Gene Ontology Enrichment Analysis (GOEA)
We use data from a 2014 Nature paper:    
[Computational analysis of cell-to-cell heterogeneity
in single-cell RNA-sequencing data reveals hidden 
subpopulations of cells
](http://www.nature.com/nbt/journal/v33/n2/full/nbt.3102.html#methods)

Note: you must have the Python package, **xlrd**, installed to run this example. 

Note: To create plots, you must have:
  * Python packages: **pyparsing**, **pydot**
  * [Graphviz](http://www.graphviz.org/) loaded and your PATH environmental variable pointing to the Graphviz bin directory.

## 1. Download Ontologies and Associations

### 1a. Download Ontologies, if necessary

In [1]:
# Get http://geneontology.org/ontology/go-basic.obo
from goatools.base import download_go_basic_obo
obo_fname = download_go_basic_obo()

  EXISTS: go-basic.obo


### 1b. Download Associations, if necessary

In [2]:
# Get ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz
from goatools.base import download_ncbi_associations
gene2go = download_ncbi_associations()

  EXISTS: gene2go


## 2. Load Ontologies, Associations and Background gene set 

### 2a. Load Ontologies

In [3]:
from goatools.obo_parser import GODag

obodag = GODag("go-basic.obo")

load obo file go-basic.obo
go-basic.obo: fmt(1.2) rel(2017-08-29) 49,075 GO Terms


### 2b. Load Associations

In [4]:
from __future__ import print_function
from goatools.associations import read_ncbi_gene2go

geneid2gos_mouse = read_ncbi_gene2go("gene2go", taxids=[10090])

print("{N:,} annotated mouse genes".format(N=len(geneid2gos_mouse)))

19,732 annotated mouse genes


### 2c. Load Background gene set
In this example, the background is all mouse protein-codinge genes

In [9]:
#from goatools.test_data.genes_NCBI_10090_ProteinCoding import GeneID2nt as GeneID2nt_mus
from genes_NCBI_10090_ProteinCoding import GeneID2nt as GeneID2nt_mus

## 3. Initialize a GOEA object
The GOEA object holds the Ontologies, Associations, and background.    
Numerous studies can then be run withough needing to re-load the above items.    
In this case, we only run one GOEA.    

In [10]:
from goatools.go_enrichment import GOEnrichmentStudy

goeaobj = GOEnrichmentStudy(
        GeneID2nt_mus.keys(), # List of mouse protein-coding genes
        geneid2gos_mouse, # geneid/GO associations
        obodag, # Ontologies
        propagate_counts = False,
        alpha = 0.05, # default significance cut-off
        methods = ['fdr_bh']) # defult multipletest correction method


fisher module not installed.  Falling back on scipy.stats.fisher_exact
18,825 out of 28,212 population items found in association


TypeError: 'GOEnrichmentStudy' object does not support indexing

## 4. Read study genes
~400 genes from the Nature paper supplemental table 4

In [306]:
import sys
import os
import pandas as pd
import mygene
from unpack_dict_to_col_pd import unpack
mg = mygene.MyGeneInfo()

drive_path = '/Volumes/Brain2017/project/'
output_path = '/Volumes/Brain2017/project/GO_term_analysis_ephys/ephys_goterms/fine/'

#read in gene symbols:
ephys_genes = pd.read_csv('/Volumes/Brain2017/project/GO_term_analysis_ephys/top_ephys/top_ephys_genes_fine.csv', index_col = 0)

#remove r squared columns:
cols = [c for c in ephys_genes.columns if c.lower()[:2] != 'r_']
ephys_symbol = ephys_genes[cols]
ephys_symbol = pd.DataFrame(ephys_symbol)

#convert gene symbols to entrezgene ids, remove NAs and save to dictionary for each for each ephys measure
output = {}
for c in ephys_symbol.columns:
    out = mg.querymany(ephys_symbol[c], scopes='symbol', fields='entrezgene', species='mouse')
    out = pd.DataFrame(out)
    out = out.dropna(subset = ['entrezgene'])
    out.entrezgene = out.entrezgene.astype(int)
    entrezgene = out.entrezgene
    #filter for protein coding genes:
    entrezgene = entrezgene[entrezgene.isin(GeneID2nt_mus.keys())].dropna()
    output[c] = entrezgene


#pd.DataFrame(d, columns=('Player', 'Team', 'Passer Rating'))

querying 1-500...done.
Finished.
50 input query terms found dup hits:
	[(u'Gm17751', 2), (u'Snord104', 2), (u'Mir3064', 2), (u'Snord100', 2), (u'Snord49b', 2), (u'4933431E
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-500...done.
Finished.
36 input query terms found dup hits:
	[(u'1700111N16Rik', 2), (u'A330076H08Rik', 2), (u'Pvt1', 2), (u'Gm10125', 2), (u'Bola2', 2), (u'Tmem
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-500...done.
Finished.
54 input query terms found dup hits:
	[(u'1700084E18Rik', 2), (u'9230105E05Rik', 2), (u'A230077H06Rik', 2), (u'C130026L21Rik', 2), (u'A430
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-500...done.
Finished.
24 input query terms found dup hits:
	[(u'5031425E22Rik', 2), (u'A230057D06Rik', 4), (u'Gpx2-ps1', 2), (u'BC039771', 2), (u'6330410L21Rik'
Pass "returnall=True" to return complete lists of duplicate 

In [303]:
output['pos_upstroke_downstroke_ratio_long_square']

out = mg.querymany(ephys_symbol['pos_upstroke_downstroke_ratio_long_square'], scopes='symbol', fields='entrezgene', species='mouse')
out = pd.DataFrame(out)
out = out.dropna(subset = ['entrezgene'])
out.entrezgene = out.entrezgene.astype(int)
entrezgene = out.entrezgene
#filter for protein coding genes:

entrezgene = entrezgene[entrezgene.isin(GeneID2nt_mus.keys())].dropna()

entrezgene

querying 1-500...done.
Finished.
69 input query terms found dup hits:
	[(u'Ftx', 2), (u'4921515E04Rik', 2), (u'Mir669h', 2), (u'Gm19619', 2), (u'6030408B16Rik', 2), (u'Tra
1 input query terms found no hit:
	[u'D630045M09Rik']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


2       12352
7       56643
8      102278
9       22171
10     246727
11      17470
15     240328
16     239420
17      21933
18      71398
26     272643
27     433938
28     232232
29     233799
30      20928
31      96935
32      69367
33      56726
35      20493
36      67455
37      94191
40     210293
46      15015
47     195046
48      13395
54      56078
57      12984
58      68177
59      93843
62      69576
        ...  
537     99470
538    242037
539     71816
540     63857
541     50796
542     23801
543     56375
544    320590
545    240776
546     13006
547     12683
549    381077
550     66435
551    620913
552     74166
553    233900
554     69080
555    229937
556     26900
559     11441
560    271375
561     21987
562    105859
563     75956
564     21974
565     11758
567    244882
568    216395
569     53328
570    237107
Name: entrezgene, dtype: int64

## 5. Run Gene Ontology Enrichment Analysis (GOEA)
You may choose to keep all results or just the significant results. In this example, we choose to keep only the significant results.

In [307]:
# 'p_' means "pvalue". 'fdr_bh' is the multipletest method we are currently using.
goea_results_sig_dict = {}
for key in output.keys():
    # 'p_' means "pvalue". 'fdr_bh' is the multipletest method we are currently using.
    #geneids_study = geneid2symbol.keys()
    geneids_study = output[key]
    goea_results_all = goeaobj.run_study(geneids_study)
    goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.05]
    goeaobj.wr_xlsx(str(output_path) + str(key) + "GOEA_resultsFine_sig.xlsx", goea_results_sig)    

    goea_results_sig_dict[key] = goea_results_sig


Calculating uncorrected p-values using fisher_scipy_stats
   428 out of    460 study items found in association
Running multitest correction: statsmodels fdr_bh
  2,476 GO terms are associated with 428 of 460 study items
  17,384 GO terms are associated with 18,825 of 28,212 population items
     24 items WROTE: /Volumes/Brain2017/project/GO_term_analysis_ephys/ephys_goterms/fine/neg_adaptationGOEA_resultsFine_sig.xlsx
Calculating uncorrected p-values using fisher_scipy_stats
   422 out of    455 study items found in association
Running multitest correction: statsmodels fdr_bh
  2,868 GO terms are associated with 422 of 455 study items
  17,384 GO terms are associated with 18,825 of 28,212 population items
     50 items WROTE: /Volumes/Brain2017/project/GO_term_analysis_ephys/ephys_goterms/fine/neg_vrestGOEA_resultsFine_sig.xlsx
Calculating uncorrected p-values using fisher_scipy_stats
   431 out of    459 study items found in association
Running multitest correction: statsmodels fdr_b

Running multitest correction: statsmodels fdr_bh
  2,532 GO terms are associated with 425 of 459 study items
  17,384 GO terms are associated with 18,825 of 28,212 population items
     17 items WROTE: /Volumes/Brain2017/project/GO_term_analysis_ephys/ephys_goterms/fine/neg_riGOEA_resultsFine_sig.xlsx


In [292]:
key


'pos_upstroke_downstroke_ratio_long_square'

In [208]:
one_record = GOEnrichmentRecord(
            GO=term,
            p_uncorrected=calc_pvalue(study_count, study_n, pop_count, pop_n),
            study_items=study_items,
            pop_items=pop_items,
            ratio_in_study=(study_count, study_n),
            ratio_in_pop=(pop_count, pop_n))
 #extract fields from georesults
n = goea_results['neg_ri'][0]
print(n.GO)
print(n.name)
print(n.NS)
print(n.study_n)
print(n.ratio_in_pop)
print(n.ratio_in_study)

SyntaxError: invalid syntax (<ipython-input-208-3c1bbfe30d4d>, line 1)

## 7. Plot all significant GO terms
Plotting all significant GO terms produces a messy spaghetti plot. Such a plot can be useful sometimes because you can open it and zoom and scroll around. But sometimes it is just too messy to be of use.

The **"{NS}"** in **"nbt3102_{NS}.png"** indicates that you will see three plots, one for "biological_process"(BP), "molecular_function"(MF), and "cellular_component"(CC)

In [65]:
from goatools.godag_plot import plot_gos, plot_results, plot_goid2goobj

plot_results("nbt3102_{NS}.png", goea_results_sig)

Exception: "dot" not found in path.

### 7a. These plots are likely to messy
The *Cellular Component* plot is the smallest plot...
![BIG CC PLOT](images/nbt3102_CC.png)

### 7b. So make a smaller sub-plot
This plot contains GOEA results:
  * GO terms colored by P-value:
    * pval < 0.005 (light red)
    * pval < 0.01 (light orange)
    * pval < 0.05 (yellow)
    * pval > 0.05 (grey) Study terms that are not statistically significant
  * GO terms with study gene counts printed. e.g., "32 genes"

In [11]:
# Plot subset starting from these significant GO terms
goid_subset = [
    'GO:0003723', # MF D04 RNA binding (32 genes)
    'GO:0044822', # MF D05 poly(A) RNA binding (86 genes)
    'GO:0003729', # MF D06 mRNA binding (11 genes)
    'GO:0019843', # MF D05 rRNA binding (6 genes)
    'GO:0003746', # MF D06 translation elongation factor activity (5 genes)
]
plot_gos("nbt3102_MF_RNA_genecnt.png", 
    goid_subset, # Source GO ids
    obodag, 
    goea_results=goea_results_all) # Use pvals for coloring


  WROTE: nbt3102_MF_RNA_genecnt.png


![RNA subplot](images/nbt3102_MF_RNA_genecnt.png)

### 7c. Add study gene Symbols to plot
e.g., *11 genes: Calr, Eef1a1, Pabpc1*

In [12]:
plot_gos("nbt3102_MF_RNA_Symbols.png", 
    goid_subset, # Source GO ids
    obodag,
    goea_results=goea_results_all, # use pvals for coloring
    # We can further configure the plot...
    id2symbol=geneid2symbol, # Print study gene Symbols, not Entrez GeneIDs
    study_items=6, # Only only 6 gene Symbols max on GO terms
    items_p_line=3, # Print 3 genes per line
    )

  WROTE: nbt3102_MF_RNA_Symbols.png


![RNA subplot](images/nbt3102_MF_RNA_Symbols.png)

Copyright (C) 2016, DV Klopfenstein, H Tang. All rights reserved.